# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


## TD10 : Mélanges de lois : Classification du rayon des tumeurs

Les données du concours A2020 sont utilisées.

## Contexte

Une tumeur est un groupe de cellules anormales qui forment une masse. Les tumeurs se développent et se comportent différemment, selon qu’elles soient cancéreuses (malignes), non cancéreuses (bénignes). Le but du concours était de prédire si une tumeur est bénigne (0) ou maligne (1) en fonction des caractéristiques suivantes récoltées par imagerie médicale : 

- radius : distance moyenne entre le centre de la tumeur et son périmètre ;
- texture : écart-type des niveaux de gris représentant l'image de la tumeur ;
- perimeter : périmètre de la tumeur ;
- area : superficie de la tumeur ;
- smoothness : variation locale normalisée en fonction du radius (indice de rugosité) ;
- compactness : perimeter^2 / area -1 (indice de compacité) ;
- symmetry : mesure de symétrie ; 
- fractal dimension : ("coastline approximation" - 1).

Dans ce TD, nous modéliserons le rayon des tumeurs par un mélange de lois normales. On s'attend à ce que le rayon des tumeurs bénignes soient distribués selon une loi normale et que le rayon des tumeurs malignes soient distribués selon une autre loi normale. On pourra donc utiliser le mélange de lois pour classer les tumeurs à l'aide d'une seule variable : son rayon.

## Objectifs du TD

Ce TD est composé de trois exercices. Le premier exercice permet d'explorer les données. Le deuxième modélise une caractéristique des tumeurs, soit la rayon, avec un mélange de deux lois normales. Le troisième exercice permet de prédire la classe de la tumeur (bénigne ou maligne) en fonction des composantes du mélange ajusté.

Plusieurs fonctions vous sont fournies pour vous faciliter la tâche.

In [ ]:
using CSV, DataFrames, Distributions, Gadfly, MLBase, Random, Statistics

In [ ]:
"""
    GMM(ω::Real, μ₀::Real, σ₀::Real, μ₁::Real, σ₁::Real)

Création d'un objet de type `UnivariateMixture` de la librairie *Distributions.jl* ayant comme densité

```math
f(y) = (1-ω) ~ \\mathcal{N}( y\\mid μ₀, σ₀²) + ω ~ \\mathcal{N}( y\\mid μ₁, σ₁²)
```
"""
function GMM(ω::Real, μ₀::Real, σ₀::Real, μ₁::Real, σ₁::Real)
    
    pd = MixtureModel(Normal[ Normal(μ₀, σ₀), Normal(μ₁, σ₁)], [1-ω, ω])
    
    return pd
    
end


"""
    componentprob(mixturemodel::UnivariateMixture, y::Real; componentindex=1, logprob=false)

Calcul de la probabilité que y provienne de la composante `componentindex` du mélange `mixturemodel`.
"""
function componentprob(mixturemodel::UnivariateMixture, y::Real; componentindex=1, logprob=false)

    fc = component(mixturemodel,componentindex)
    
    lp = log(probs(mixturemodel)[componentindex]) + logpdf(fc,y) - logpdf(mixturemodel, y)
    
    if logprob
        return lp
    else
        return exp(lp)
    end
    
end

"""
    _emstep(pd::MixtureModel,y)

Réalisation d'une itération de l'algorithme EM à partir du mélange `pd` avec les données `y`.

#### Détails
La fonction met à jour les paramètres de la distribution `pd` avec les estimations améliorées.
"""
function _emstep(pd::MixtureModel,y)
    
    n = length(y)
    
    f₁ = component(pd, 2)
    ω = probs(pd)[2]
    
    lp₁ = log(ω) .+ logpdf.(f₁,y) - logpdf.(pd, y)
    p₁ = exp.(lp₁)
    
    ω̂ = sum(p₁)/n
    
    p₀ = 1 .- p₁
    
    μ̂₀ = sum( p₀.* y) / sum(p₀)
    
    σ̂₀² = sum( p₀.* (y .- μ̂₀).^2 ) / sum(p₀)
    
    μ̂₁ = sum( p₁.* y) / sum(p₁)
    
    σ̂₁² = sum( p₁.* (y .- μ̂₁).^2 ) / sum(p₁)
    
    fd = GMM(ω̂, μ̂₀, sqrt(σ̂₀²), μ̂₁, sqrt(σ̂₁²))
    
    return fd
    
end

"""
    GMMemfit(y::Vector{<:Real} ; initialvalue::Vector{<:Real}=Float64[], maxiter::Int=1000, tol::Real=2*eps())

Calcul des estimateurs du maximum de la vraisemblance d'un mélange de lois normales avec l'algorithme EM.
"""
function GMMemfit(y::Vector{<:Real} ; initialvalue::Vector{<:Real}=Float64[], maxiter::Int=1000, tol::Real=2*eps())
    
    if isempty(initialvalue)
        
        n = length(y)
        
        ind = (1:n) .< n/2
        
        y₀ = y[ind]
        y₁ = y[.!(ind)]
        
        initialvalue = [.5, mean(y₀), std(y₀), mean(y₁), std(y₁)]
        
    end
    
    pd = GMM(initialvalue...)
    
    iter = 1
    err = 1
    
    while (err > tol) & (iter < maxiter)
       
        fd = _emstep(pd,y)
        
        err = abs(loglikelihood(fd,y) - loglikelihood(pd,y))
        
        pd = fd
        
        iter +=1
        
    end
    
    μ₀ = mean(components(pd)[1])
    μ₁ = mean(components(pd)[2])

    if μ₀ > μ₁
        μ₀ = mean(components(pd)[2])
        σ₀ = std(components(pd)[2])
        μ₁ = mean(components(pd)[1])
        σ₁ = std(components(pd)[1])
        ω = probs(pd)[1]

        pd = GMM(ω, μ₀, σ₀, μ₁, σ₁)

    end
    
    
    
    if iter == maxiter
        println("Convergence not reached in $maxiter iterations")
    else
        println("Convergence reached in $iter iterations")
    end
    
 return pd
    
end

"""
    histplot(fd::UnivariateMixture, y::Vector{<:Real})

Trace le mélange de lois `fd` superposé à l'histogramme des données `y`.
"""
function histplot(fd::UnivariateMixture, y::Vector{<:Real})
   
    @assert length(components(fd)) == 2 "the function is optimized for a mixture of two components."
    
    nbin = floor(Int,sqrt(length(y)))
    opacity = repeat([0.75, 0.85], outer=nbin)
    
    xmin = minimum(y)
    xmax = maximum(y)
       
    plot(Guide.ylabel("densité"), Guide.xlabel("y"), Coord.cartesian(xmin=xmin, xmax=xmax),
        layer(x -> pdf(fd, x), xmin , xmax, Theme(default_color="black")),
        layer(x -> probs(fd)[1]*pdf(components(fd)[1], x), xmin , xmax, Geom.line, Theme(default_color="gold2")),
        layer(x -> probs(fd)[2]*pdf(components(fd)[2], x), xmin , xmax, Theme(default_color="red")),
        layer(x=y, alpha=opacity, Geom.histogram(position=:identity, bincount = nbin, density=true)),
    )
        
end

# Chargement des données

On charge les données de l'ensemble d'entraînement du concours, soit les caractéristiques des mesurées des tumeurs malignes et bénignes.

In [ ]:
ENV["COLUMNS"] = 1000;  # pour voir toutes les colonnes

data = CSV.read("train.csv", DataFrame)
first(data,5)

### Partitionnement des données en ensemble d'entraînement et de validation

In [ ]:
Random.seed!(3302)
train_id = sample(1:nrow(data), round(Int, .8*nrow(data)), ordered=true, replace=false)
valid_id = setdiff(1:nrow(data), train_id)

train = data[train_id,:]
valid = data[valid_id,:];

# Exercice 1 - Analyse exploratoire partielle

Analyse exploratoire partielle de l'ensemble d'entraînement.

### (a) Tracez l'histogramme illustrant le rayon des tumeurs  

In [ ]:
# Pour définir le nombre de bins
nbin = floor(sqrt(length(train.radius)))

# Pour régler les dimensions du graphique
set_default_plot_size(12cm, 8cm)

plot(train, x=:radius, Geom.histogram(bincount=nbin))  # avec un DataFrame

#plot(x=train.radius, Geom.histogram(bincount=nbin))  # avec un vecteur 

# Ici, on peut presque déjà apercevoir deux groupes, un autour de 12-13 et l'autre vers 18-20

### (b) Calculez la proportion de tumeurs malignes

In [ ]:
# si maligne, train.diagnosis = 1
# si bénigne, train.diagnosis = 0

mean(train.diagnosis)  # version simple

In [ ]:
count(train.diagnosis .== 1) / length(train.diagnosis)  # La version longue

### (c) Calculez la moyenne et l'écart-type du rayon en fonction de la classe de la tumeur

Vous pouvez utiliser les fonctions `groupby()` et `combine()`pour des opérations rapides.

In [ ]:
df = combine(groupby(train, :diagnosis), # on groupe les données selon :diagnosis,
    
        :radius => mean => :moyenne,  # colonne radius, on calcule la moy. et on la passe dans une nouv. col.
       
        :radius => std => :ecarttype # puis on fait la même chose avec l'écart-type
    
     )

### (d) Tracez des diagrammes en boîtes des rayons en fonction de la classe des tumeurs

Utilisez la géométrie `Geom.boxplot()` de *Gadfly.jl*.

In [ ]:
plot(train, x=:diagnosis, y=:radius,  Geom.boxplot())

# Exercice 2 - Modélisation du rayon par un mélange de lois

Modélisation du rayon des tumeurs (`:radius`) avec un mélange de lois de deux lois normales.



### (a) Estimez les paramètres du mélange de lois normales modélisant le rayon des tumeurs

Utilisez l'algorithme EM pour trouver les estimations du maximum de la vraisemblance.

Vous pouvez utiliser la fonction `GMMemfit()` fournie qui renvoie un objet de type `GMM` (aussi fourni).

In [ ]:
fd = GMMemfit(train.radius)  # la fonction prend le vecteur des rayons en entrée

### (b) Tracez le mélange de lois superposé à l'histogramme des données

Vouz pouvez utiliser la fonction `histplot()` fournie.

In [ ]:
histplot(fd, train.radius)  # beau mélange de lois normales ici

### (c) Décrivez les composantes obtenues

**Note :** L'objet `GMM` est de type `UnivariateMixture` de la librairie *Distributions.jl*. On peut utiliser les méthodes standards de la librairie tel `pdf()` et `logpdf()` sur le type `UnivariateMixture`. Les méthodes suivantes sont particulières au type `UnivariateMixture` :
- `probs(fd::UnivariateMixture)` : retourne le vecteur des poids de chacune des composantes ;
- `component(fd::UnivariateMixture, k)` : retourne un objet de type `Distribution` correspondant à la composante k.

In [ ]:
# Ici, la composante 1 a un poids de 68%  et la composante 2, 32%
probs(fd)

In [ ]:
# Pour la composante 1, la moyenne et l'écart-type de la loi Normale sont :
component(fd, 1)

In [ ]:
# Pour la composante 2, la moyenne et l'écart-type de la loi Normale sont :
component(fd, 2)

### (d) Est-ce que les paramètres obtenus sont cohérents avec les questions (b) et (c) de l'exercice 1 ?

In [ ]:
df  # Oui ! Ça ressemble beaucoup à ce qu'on avait obtenu....

# Exercice 3 - Classification des tumeurs

Utilisation du mélange de lois pour classifier les tumeurs entre bénignes et malignes.

### (a) Calculez les probabilités que les observations proviennent de la 2e composante du mélange.

Calculez ces probabilités pour les observations des l'ensemble d'entraînement.

Vous pouvez utiliser la fonction `componentprob()` fournie pour calculer ces probabilités.

In [ ]:
p = componentprob.(fd, train.radius, componentindex=2);

              #  ↑ . pour le faire sur tout le vecteur de rayons train.radius

In [ ]:
p  # on obtient un vecteur de prob de 364 éléments 

### (b) Prédisez les classes des tumeurs en fonction des probabilités calculées aux numéros précédents.

In [ ]:
# j'initialise un vecteur de zéro de la même taille que le nombre de tumeurs (364)
ŷ = Int.(zeros(length(train.diagnosis))) ; 

# si la prob d'apartenir à la composant 2 est supérieure à 0.5, je classe la tumeur comme étant maligne
ŷ[p .> 0.5] .= 1 

### (c) Calculez les qualités des prédictions obtenues

In [ ]:
y = train.diagnosis  # le vrai diagnostic 

r = roc(y, ŷ)  # on calcule la ROC avec la fonction roc() de MLBase

In [ ]:
println("Sur l'ensemble d'entraînement")
println("- Nombre de tumeurs malignes bien identifiées : ",r.tp)
println("- Nombre de tumeurs bénignes bien identifiées : ",r.tn)
println("- Nombre de tumeurs malignes identifiées comme bénignes : ",r.fn)
println("- Nombre de tumeurs bénignes identifiées comme malignes : ",r.fp)
println("- Le score F₁ est de : ", round(f1score(r), digits=4))

### (d) Répétez les étapes pour obtenir les prédictions sur l'ensemble de validation

In [ ]:
# on calcule maitenant le prob d'appartenir à la composante 2 de fd (fitté sur train) avec les rayons de valid
p = componentprob.(fd, valid.radius, componentindex=2); 

In [ ]:
# j'initialise un vecteur de zéro de la même taille que le nombre de tumeurs (de valid)
ŷ = Int.(zeros(length(valid.diagnosis)))  

# si la prob d'apartenir à la composant 2 est supérieure à 0.5, je classe la tumeur comme étant maligne
ŷ[p .> 0.5] .= 1 ;

In [ ]:
y = valid.diagnosis  # le vrai diagnostic (de valid)

r = roc(y, ŷ)  # on calcule la ROC avec la fonction roc() de MLBase

In [ ]:
println("Sur l'ensemble d'entraînement")
println("- Nombre de tumeurs malignes bien identifiées : ",r.tp)
println("- Nombre de tumeurs bénignes bien identifiées : ",r.tn)
println("- Nombre de tumeurs malignes identifiées comme bénignes : ",r.fn)
println("- Nombre de tumeurs bénignes identifiées comme malignes : ",r.fp)
println("- Le score F₁ est de : ", round(f1score(r), digits=4))